# 🤖 KI-Integration in der Produktionsplanung

**Intelligente Produktionsplanung und KI**  
Universität Stuttgart  
Master Maschinenbau / Technologiemanagement

---

## Lernziele

Nach Bearbeitung dieses Notebooks können Sie:

1. Eine **End-to-End Machine Learning Pipeline** für Produktionsdaten aufbauen
2. **Supervised Learning Modelle** (Random Forest, Gradient Boosting) anwenden und bewerten
3. **Feature Importance** analysieren und für Prozessoptimierung nutzen
4. **Szenario-Analysen** mit trainierten Modellen durchführen
5. **Industrie 4.0 Trends** und deren KI-Anwendungen einordnen
6. Eine **Implementierungsstrategie** für KI in der Produktion entwickeln

---

## 1. Einführung: KI in der Produktionsplanung

### Warum KI in der Produktion?

Die **vierte industrielle Revolution (Industrie 4.0)** transformiert die Fertigungsindustrie durch:

| Treiber | Beschreibung |
|---------|-------------|
| **Datenexplosion** | IoT-Sensoren generieren Terabytes an Produktionsdaten |
| **Rechenleistung** | Cloud Computing und GPUs ermöglichen komplexe Modelle |
| **Algorithmen** | Fortschritte in Machine Learning und Deep Learning |
| **Wettbewerb** | Kostendruck und Qualitätsanforderungen steigen |

### KI-Anwendungsfelder in der Produktion

```
┌─────────────────────────────────────────────────────────────────┐
│                    KI in der Produktion                         │
├─────────────────┬─────────────────┬─────────────────────────────┤
│   Prognose      │   Optimierung   │      Automatisierung        │
├─────────────────┼─────────────────┼─────────────────────────────┤
│ • Nachfrage     │ • Scheduling    │ • Qualitätskontrolle        │
│ • Qualität      │ • Ressourcen    │ • Predictive Maintenance    │
│ • Ausfälle      │ • Energie       │ • Robotersteuerung          │
│ • Kosten        │ • Routen        │ • Anomalieerkennung         │
└─────────────────┴─────────────────┴─────────────────────────────┘
```

### Machine Learning Grundkonzepte

| Lerntyp | Beschreibung | Beispiel in Produktion |
|---------|-------------|------------------------|
| **Supervised** | Lernen aus gelabelten Daten | Qualitätsprognose aus Prozessparametern |
| **Unsupervised** | Muster in ungelabelten Daten | Anomalieerkennung in Sensordaten |
| **Reinforcement** | Lernen durch Belohnung | Robotersteuerung, Scheduling |

In [ ]:
# ===================================================================
# Imports und Konfiguration
# ===================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Matplotlib Konfiguration
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 11
plt.style.use('seaborn-v0_8-whitegrid')

print("✅ Alle Bibliotheken erfolgreich geladen!")
print("\n📦 Verwendete Pakete:")
print(f"   - NumPy: {np.__version__}")
print(f"   - Pandas: {pd.__version__}")
print(f"   - Scikit-learn: Ensemble-Methoden, Preprocessing, Metriken")

---

## 2. Fallbeispiel: Intelligente Fertigungssteuerung

### Problemstellung

Ein **Fertigungsunternehmen** möchte KI einsetzen, um:

1. **Fertigungszeiten** vorherzusagen und Kapazitäten zu planen
2. **Qualität** zu prognostizieren und Ausschuss zu reduzieren
3. **Energieverbrauch** zu optimieren und Kosten zu senken

### Datenbasis

Wir generieren einen **realistischen Produktionsdatensatz** mit folgenden Merkmalen:

| Feature-Kategorie | Features | Beschreibung |
|-------------------|----------|-------------|
| **Produkt** | Produkttyp | Elektronik, Mechanik, Optik, etc. |
| **Prozess** | Maschine, Schicht | Ressourcenzuordnung |
| **Umgebung** | Temperatur, Luftfeuchtigkeit | Umgebungsbedingungen |
| **Qualität** | Materialqualität | Eingangsmaterial-Güte |
| **Personal** | Operator-Erfahrung | Qualifikation |
| **Wartung** | Tage seit Wartung | Maschinenzustand |

In [ ]:
# ===================================================================
# Datensatz für KI-Anwendungen generieren
# ===================================================================

def generiere_produktionsdaten(n_samples=2000, seed=42):
    """
    Generiert einen realistischen Produktionsdatensatz mit kausalen Zusammenhängen.
    
    Parameter:
    ----------
    n_samples : int
        Anzahl der Datenpunkte
    seed : int
        Random Seed für Reproduzierbarkeit
        
    Returns:
    --------
    DataFrame : Produktionsdatensatz
    """
    np.random.seed(seed)
    
    # Kategorische Variablen
    produkttypen = ['Elektronik', 'Mechanik', 'Optik', 'Hybrid']
    maschinen = ['CNC_A', 'CNC_B', 'Robot_1', 'Robot_2', '3D_Drucker']
    schichten = ['Früh', 'Spät', 'Nacht']
    
    # Faktoren für kausale Zusammenhänge
    komplexitaet = {'Elektronik': 1.2, 'Mechanik': 1.0, 'Optik': 1.6, 'Hybrid': 1.4}
    maschinen_effizienz = {'CNC_A': 1.1, 'CNC_B': 1.0, 'Robot_1': 1.3, 'Robot_2': 1.2, '3D_Drucker': 0.9}
    schicht_faktor = {'Früh': 1.0, 'Spät': 0.95, 'Nacht': 0.88}
    energie_faktor = {'CNC_A': 0.8, 'CNC_B': 1.0, 'Robot_1': 0.6, 'Robot_2': 0.7, '3D_Drucker': 1.2}
    
    daten = []
    
    for i in range(n_samples):
        # Zufällige Auswahl kategorischer Variablen
        produkttyp = np.random.choice(produkttypen)
        maschine = np.random.choice(maschinen)
        schicht = np.random.choice(schichten)
        
        # Numerische Features
        temperatur = np.random.normal(22, 2.5)
        luftfeuchtigkeit = np.random.normal(45, 8)
        materialqualitaet = np.random.uniform(6.5, 10)
        operator_erfahrung = np.random.exponential(4) + 0.5
        tage_seit_wartung = np.random.poisson(12)
        
        # === Zielgrößen mit kausalen Zusammenhängen ===
        
        # 1. Fertigungszeit (Minuten)
        basis_zeit = 45 * komplexitaet[produkttyp]
        zeit = basis_zeit / maschinen_effizienz[maschine] / schicht_faktor[schicht]
        zeit *= (1 + (temperatur - 22) * 0.008)  # Temperatureinfluss
        zeit *= (1 + tage_seit_wartung * 0.004)  # Wartungseinfluss
        zeit *= np.random.normal(1, 0.08)  # Zufallsvariation
        fertigungszeit = max(zeit, 15)
        
        # 2. Qualitätsscore (0-1)
        qualitaet = 0.92
        qualitaet += (materialqualitaet - 8) * 0.02  # Materialeinfluss
        qualitaet += min(operator_erfahrung / 15, 0.05)  # Erfahrungseinfluss
        qualitaet -= tage_seit_wartung * 0.002  # Wartungseinfluss
        qualitaet *= schicht_faktor[schicht]  # Schichteinfluss
        qualitaet += np.random.normal(0, 0.03)
        qualitaetsscore = np.clip(qualitaet, 0.70, 0.99)
        
        # 3. Energieverbrauch (kWh)
        basis_energie = 4 * komplexitaet[produkttyp]
        energie = basis_energie * energie_faktor[maschine] * (fertigungszeit / 60)
        energie *= np.random.normal(1, 0.1)
        energieverbrauch = max(energie, 0.5)
        
        # 4. Ausschusswahrscheinlichkeit
        ausschuss = 0.02 + (1 - qualitaetsscore) * 0.15
        ausschuss += tage_seit_wartung * 0.001
        ausschuss = np.clip(ausschuss + np.random.exponential(0.01), 0, 0.20)
        
        daten.append({
            'Produkttyp': produkttyp,
            'Maschine': maschine,
            'Schicht': schicht,
            'Temperatur_C': round(temperatur, 1),
            'Luftfeuchtigkeit_pct': round(luftfeuchtigkeit, 1),
            'Materialqualitaet': round(materialqualitaet, 1),
            'Operator_Erfahrung_Jahre': round(min(operator_erfahrung, 25), 1),
            'Tage_seit_Wartung': int(tage_seit_wartung),
            'Fertigungszeit_min': round(fertigungszeit, 1),
            'Qualitaetsscore': round(qualitaetsscore, 3),
            'Energieverbrauch_kWh': round(energieverbrauch, 2),
            'Ausschussrate': round(ausschuss, 4)
        })
    
    return pd.DataFrame(daten)

# Datensatz generieren
df = generiere_produktionsdaten(n_samples=2000)

print("📊 PRODUKTIONSDATENSATZ")
print("=" * 65)
print(f"\nAnzahl Datenpunkte: {len(df)}")
print(f"Anzahl Features: {len(df.columns)}")

print("\n📋 FEATURE-ÜBERSICHT:")
print("-" * 65)
print("Kategorische Features:")
for col in ['Produkttyp', 'Maschine', 'Schicht']:
    print(f"   {col}: {df[col].unique().tolist()}")

print("\nNumerische Features:")
num_cols = ['Temperatur_C', 'Luftfeuchtigkeit_pct', 'Materialqualitaet', 
           'Operator_Erfahrung_Jahre', 'Tage_seit_Wartung']
print(df[num_cols].describe().round(2).to_string())

print("\n🎯 ZIELGRÖSSEN:")
ziel_cols = ['Fertigungszeit_min', 'Qualitaetsscore', 'Energieverbrauch_kWh', 'Ausschussrate']
print(df[ziel_cols].describe().round(3).to_string())

In [ ]:
# ===================================================================
# Explorative Datenanalyse
# ===================================================================

def explorative_analyse(df):
    """
    Führt eine explorative Datenanalyse durch.
    """
    
    fig, axes = plt.subplots(2, 3, figsize=(16, 10))
    
    # 1. Fertigungszeit nach Produkttyp
    ax1 = axes[0, 0]
    df.boxplot(column='Fertigungszeit_min', by='Produkttyp', ax=ax1)
    ax1.set_title('Fertigungszeit nach Produkttyp')
    ax1.set_xlabel('Produkttyp')
    ax1.set_ylabel('Fertigungszeit (min)')
    plt.suptitle('')  # Entferne automatischen Titel
    
    # 2. Qualität nach Maschine
    ax2 = axes[0, 1]
    df.boxplot(column='Qualitaetsscore', by='Maschine', ax=ax2)
    ax2.set_title('Qualitätsscore nach Maschine')
    ax2.set_xlabel('Maschine')
    ax2.set_ylabel('Qualitätsscore')
    ax2.tick_params(axis='x', rotation=45)
    
    # 3. Energieverbrauch nach Schicht
    ax3 = axes[0, 2]
    df.boxplot(column='Energieverbrauch_kWh', by='Schicht', ax=ax3)
    ax3.set_title('Energieverbrauch nach Schicht')
    ax3.set_xlabel('Schicht')
    ax3.set_ylabel('Energieverbrauch (kWh)')
    
    # 4. Korrelation: Wartung vs. Qualität
    ax4 = axes[1, 0]
    ax4.scatter(df['Tage_seit_Wartung'], df['Qualitaetsscore'], alpha=0.3, s=20)
    z = np.polyfit(df['Tage_seit_Wartung'], df['Qualitaetsscore'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(df['Tage_seit_Wartung'].min(), df['Tage_seit_Wartung'].max(), 100)
    ax4.plot(x_line, p(x_line), 'r-', linewidth=2, label='Trendlinie')
    ax4.set_xlabel('Tage seit Wartung')
    ax4.set_ylabel('Qualitätsscore')
    ax4.set_title('Wartungseinfluss auf Qualität')
    ax4.legend()
    
    # 5. Korrelation: Erfahrung vs. Qualität
    ax5 = axes[1, 1]
    ax5.scatter(df['Operator_Erfahrung_Jahre'], df['Qualitaetsscore'], alpha=0.3, s=20)
    z = np.polyfit(df['Operator_Erfahrung_Jahre'], df['Qualitaetsscore'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(0, df['Operator_Erfahrung_Jahre'].max(), 100)
    ax5.plot(x_line, p(x_line), 'r-', linewidth=2, label='Trendlinie')
    ax5.set_xlabel('Operator Erfahrung (Jahre)')
    ax5.set_ylabel('Qualitätsscore')
    ax5.set_title('Erfahrungseinfluss auf Qualität')
    ax5.legend()
    
    # 6. Korrelationsmatrix (numerische Features)
    ax6 = axes[1, 2]
    korr_cols = ['Temperatur_C', 'Materialqualitaet', 'Operator_Erfahrung_Jahre', 
                'Tage_seit_Wartung', 'Fertigungszeit_min', 'Qualitaetsscore']
    korr_matrix = df[korr_cols].corr()
    im = ax6.imshow(korr_matrix, cmap='RdBu_r', aspect='auto', vmin=-1, vmax=1)
    ax6.set_xticks(range(len(korr_cols)))
    ax6.set_yticks(range(len(korr_cols)))
    ax6.set_xticklabels([c.replace('_', '\n') for c in korr_cols], fontsize=8)
    ax6.set_yticklabels([c.replace('_', '\n') for c in korr_cols], fontsize=8)
    ax6.set_title('Korrelationsmatrix')
    plt.colorbar(im, ax=ax6, shrink=0.8)
    
    plt.tight_layout()
    plt.show()

print("📊 EXPLORATIVE DATENANALYSE")
print("=" * 50)
explorative_analyse(df)

print("\n💡 ERKENNTNISSE AUS DER EDA:")
print("   • Optik-Produkte haben die längste Fertigungszeit")
print("   • Roboter erzielen höhere Qualitätsscores als manuelle Prozesse")
print("   • Qualität sinkt mit zunehmender Zeit seit letzter Wartung")
print("   • Erfahrene Operatoren erzielen tendenziell bessere Qualität")

---

## 3. Machine Learning Pipeline

### Überblick ML-Workflow

```
┌──────────────┐    ┌──────────────┐    ┌──────────────┐    ┌──────────────┐
│   Rohdaten   │───▶│  Preprocessing│───▶│   Training   │───▶│  Evaluation  │
└──────────────┘    └──────────────┘    └──────────────┘    └──────────────┘
                           │                    │                    │
                    • Encoding         • Train/Test Split   • MAE, RMSE, R²
                    • Skalierung       • Cross-Validation   • Feature Importance
                    • Feature Eng.     • Hyperparameter     • Residuenanalyse
```

### Verwendete Algorithmen

| Algorithmus | Beschreibung | Stärken |
|-------------|-------------|--------|
| **Random Forest** | Ensemble aus Entscheidungsbäumen | Robust, wenig Hyperparameter, Feature Importance |
| **Gradient Boosting** | Sequentielles Ensemble | Höhere Genauigkeit, Overfitting-Risiko |

In [ ]:
# ===================================================================
# Machine Learning Pipeline Klasse
# ===================================================================

class ProduktionsML:
    """
    Machine Learning Pipeline für Produktionsdaten.
    
    Diese Klasse kapselt den gesamten ML-Workflow:
    - Datenvorverarbeitung (Encoding, Skalierung)
    - Modelltraining und -vergleich
    - Evaluation und Visualisierung
    - Vorhersagen für neue Daten
    """
    
    def __init__(self, df):
        """
        Initialisiert die Pipeline.
        
        Parameter:
        ----------
        df : DataFrame
            Produktionsdatensatz
        """
        self.df = df.copy()
        self.label_encoders = {}
        self.scaler = StandardScaler()
        self.models = {}
        self.feature_names = None
        
    def prepare_features(self):
        """
        Bereitet die Feature-Matrix vor.
        
        Returns:
        --------
        DataFrame : Vorverarbeitete Features
        """
        # Feature-Spalten definieren
        self.feature_cols = ['Produkttyp', 'Maschine', 'Schicht',
                            'Temperatur_C', 'Luftfeuchtigkeit_pct',
                            'Materialqualitaet', 'Operator_Erfahrung_Jahre',
                            'Tage_seit_Wartung']
        
        X = self.df[self.feature_cols].copy()
        
        # Kategorische Variablen encodieren
        cat_cols = ['Produkttyp', 'Maschine', 'Schicht']
        for col in cat_cols:
            self.label_encoders[col] = LabelEncoder()
            X[col] = self.label_encoders[col].fit_transform(X[col])
        
        self.feature_names = list(X.columns)
        return X
    
    def train_and_evaluate(self, target_col, test_size=0.2):
        """
        Trainiert und evaluiert verschiedene Modelle.
        
        Parameter:
        ----------
        target_col : str
            Name der Zielvariable
        test_size : float
            Anteil der Testdaten
            
        Returns:
        --------
        dict : Ergebnisse für alle Modelle
        """
        
        print(f"\n{'='*60}")
        print(f"🎯 MODELLTRAINING: {target_col}")
        print(f"{'='*60}")
        
        # Daten vorbereiten
        X = self.prepare_features()
        y = self.df[target_col]
        
        # Train-Test Split
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=42
        )
        
        print(f"\n📊 Datensplit:")
        print(f"   Training: {len(X_train)} Samples")
        print(f"   Test:     {len(X_test)} Samples")
        
        # Modelle definieren
        models = {
            'Random Forest': RandomForestRegressor(
                n_estimators=100, max_depth=15, min_samples_split=5,
                random_state=42, n_jobs=-1
            ),
            'Gradient Boosting': GradientBoostingRegressor(
                n_estimators=100, max_depth=5, learning_rate=0.1,
                random_state=42
            )
        }
        
        results = {}
        
        print(f"\n📈 MODELLVERGLEICH:")
        print("-" * 60)
        print(f"{'Modell':<20} {'MAE':>10} {'RMSE':>10} {'R²':>10} {'CV-MAE':>10}")
        print("-" * 60)
        
        for name, model in models.items():
            # Training
            model.fit(X_train, y_train)
            
            # Vorhersagen
            y_pred_test = model.predict(X_test)
            
            # Metriken
            mae = mean_absolute_error(y_test, y_pred_test)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
            r2 = r2_score(y_test, y_pred_test)
            
            # Cross-Validation
            cv_scores = cross_val_score(model, X_train, y_train, cv=5,
                                       scoring='neg_mean_absolute_error')
            cv_mae = -cv_scores.mean()
            
            print(f"{name:<20} {mae:>10.4f} {rmse:>10.4f} {r2:>10.3f} {cv_mae:>10.4f}")
            
            results[name] = {
                'model': model,
                'mae': mae,
                'rmse': rmse,
                'r2': r2,
                'cv_mae': cv_mae,
                'y_test': y_test,
                'y_pred': y_pred_test,
                'feature_importance': model.feature_importances_
            }
        
        # Bestes Modell auswählen
        best_model_name = min(results, key=lambda x: results[x]['mae'])
        print("-" * 60)
        print(f"\n🏆 Bestes Modell: {best_model_name} (MAE: {results[best_model_name]['mae']:.4f})")
        
        # Bestes Modell speichern
        self.models[target_col] = results[best_model_name]['model']
        
        return results, best_model_name
    
    def plot_results(self, results, target_col, best_model_name):
        """
        Visualisiert die Modellergebnisse.
        """
        
        best = results[best_model_name]
        
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        # 1. Vorhersage vs. Tatsächlich
        ax1 = axes[0, 0]
        ax1.scatter(best['y_test'], best['y_pred'], alpha=0.5, s=30)
        lims = [min(best['y_test'].min(), best['y_pred'].min()),
               max(best['y_test'].max(), best['y_pred'].max())]
        ax1.plot(lims, lims, 'r--', linewidth=2, label='Perfekte Vorhersage')
        ax1.set_xlabel('Tatsächliche Werte')
        ax1.set_ylabel('Vorhergesagte Werte')
        ax1.set_title(f'{best_model_name}: Vorhersage vs. Tatsächlich')
        ax1.legend()
        ax1.text(0.05, 0.95, f"R² = {best['r2']:.3f}\nMAE = {best['mae']:.3f}",
                transform=ax1.transAxes, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        # 2. Residuen-Plot
        ax2 = axes[0, 1]
        residuen = best['y_test'].values - best['y_pred']
        ax2.scatter(best['y_pred'], residuen, alpha=0.5, s=30)
        ax2.axhline(y=0, color='r', linestyle='--', linewidth=2)
        ax2.set_xlabel('Vorhergesagte Werte')
        ax2.set_ylabel('Residuen')
        ax2.set_title('Residuen-Analyse')
        
        # 3. Feature Importance
        ax3 = axes[1, 0]
        importance = best['feature_importance']
        sorted_idx = np.argsort(importance)
        ax3.barh([self.feature_names[i] for i in sorted_idx], 
                importance[sorted_idx], color='steelblue')
        ax3.set_xlabel('Feature Importance')
        ax3.set_title('Feature Wichtigkeit')
        
        # 4. Fehlerhistogramm
        ax4 = axes[1, 1]
        ax4.hist(residuen, bins=30, edgecolor='black', alpha=0.7)
        ax4.axvline(x=0, color='r', linestyle='--', linewidth=2)
        ax4.set_xlabel('Fehler (Residuen)')
        ax4.set_ylabel('Häufigkeit')
        ax4.set_title('Fehlerverteilung')
        ax4.text(0.95, 0.95, f"Std: {np.std(residuen):.3f}",
                transform=ax4.transAxes, ha='right', va='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        plt.suptitle(f'Modellevaluation: {target_col}', fontsize=14, fontweight='bold', y=1.02)
        plt.tight_layout()
        plt.show()
        
        # Feature Importance Interpretation
        print(f"\n📊 FEATURE IMPORTANCE für {target_col}:")
        print("-" * 40)
        for idx in sorted_idx[::-1][:5]:
            print(f"   {self.feature_names[idx]:<25}: {importance[idx]:.3f}")
    
    def predict(self, target_col, input_data):
        """
        Macht Vorhersagen für neue Daten.
        
        Parameter:
        ----------
        target_col : str
            Zielvariable
        input_data : dict
            Dictionary mit Feature-Werten
            
        Returns:
        --------
        float : Vorhersagewert
        """
        if target_col not in self.models:
            raise ValueError(f"Kein trainiertes Modell für {target_col}")
        
        # Input vorbereiten
        X_new = pd.DataFrame([input_data])[self.feature_cols]
        
        # Kategorische Variablen encodieren
        for col in ['Produkttyp', 'Maschine', 'Schicht']:
            X_new[col] = self.label_encoders[col].transform(X_new[col])
        
        return self.models[target_col].predict(X_new)[0]

# Pipeline initialisieren
ml_pipeline = ProduktionsML(df)
print("✅ ML-Pipeline initialisiert")

---

## 4. Modelltraining und Evaluation

Wir trainieren Modelle für drei Zielgrößen:
1. **Fertigungszeit** - für Kapazitätsplanung
2. **Qualitätsscore** - für Qualitätsmanagement
3. **Energieverbrauch** - für Nachhaltigkeitsoptimierung

In [ ]:
# ===================================================================
# Modell 1: Fertigungszeit-Prognose
# ===================================================================

results_zeit, best_zeit = ml_pipeline.train_and_evaluate('Fertigungszeit_min')
ml_pipeline.plot_results(results_zeit, 'Fertigungszeit_min', best_zeit)

In [ ]:
# ===================================================================
# Modell 2: Qualitätsprognose
# ===================================================================

results_qual, best_qual = ml_pipeline.train_and_evaluate('Qualitaetsscore')
ml_pipeline.plot_results(results_qual, 'Qualitaetsscore', best_qual)

In [ ]:
# ===================================================================
# Modell 3: Energieverbrauch-Prognose
# ===================================================================

results_energie, best_energie = ml_pipeline.train_and_evaluate('Energieverbrauch_kWh')
ml_pipeline.plot_results(results_energie, 'Energieverbrauch_kWh', best_energie)

---

## 5. Szenario-Analyse und Optimierung

Mit den trainierten Modellen können wir **Was-wäre-wenn-Analysen** durchführen und **Optimierungsempfehlungen** ableiten.

In [ ]:
# ===================================================================
# Szenario-Analyse
# ===================================================================

def szenario_analyse(ml_pipeline):
    """
    Führt eine Szenario-Analyse mit verschiedenen Parameterkonfigurationen durch.
    """
    
    print("\n🔍 SZENARIO-ANALYSE")
    print("=" * 65)
    
    # Basis-Szenario
    basis = {
        'Produkttyp': 'Elektronik',
        'Maschine': 'CNC_B',
        'Schicht': 'Früh',
        'Temperatur_C': 22.0,
        'Luftfeuchtigkeit_pct': 45.0,
        'Materialqualitaet': 7.5,
        'Operator_Erfahrung_Jahre': 3.0,
        'Tage_seit_Wartung': 15
    }
    
    # Optimierte Szenarien
    szenarien = {
        'Basis': basis.copy(),
        'Bessere Maschine': {**basis, 'Maschine': 'Robot_1'},
        'Besseres Material': {**basis, 'Materialqualitaet': 9.5},
        'Erfahrener Operator': {**basis, 'Operator_Erfahrung_Jahre': 10.0},
        'Frische Wartung': {**basis, 'Tage_seit_Wartung': 2},
        'Voll optimiert': {
            **basis,
            'Maschine': 'Robot_1',
            'Materialqualitaet': 9.5,
            'Operator_Erfahrung_Jahre': 10.0,
            'Tage_seit_Wartung': 2
        }
    }
    
    # Vorhersagen für alle Szenarien
    ergebnisse = []
    
    for name, params in szenarien.items():
        zeit = ml_pipeline.predict('Fertigungszeit_min', params)
        qual = ml_pipeline.predict('Qualitaetsscore', params)
        energie = ml_pipeline.predict('Energieverbrauch_kWh', params)
        
        ergebnisse.append({
            'Szenario': name,
            'Fertigungszeit': zeit,
            'Qualität': qual,
            'Energie': energie
        })
    
    df_szenarien = pd.DataFrame(ergebnisse)
    
    # Verbesserungen berechnen
    basis_vals = df_szenarien[df_szenarien['Szenario'] == 'Basis'].iloc[0]
    
    print("\n📊 ERGEBNISSE:")
    print("-" * 70)
    print(f"{'Szenario':<20} {'Zeit (min)':>12} {'Qualität':>12} {'Energie (kWh)':>14}")
    print("-" * 70)
    
    for _, row in df_szenarien.iterrows():
        print(f"{row['Szenario']:<20} {row['Fertigungszeit']:>12.1f} {row['Qualität']:>12.3f} {row['Energie']:>14.2f}")
    
    # Verbesserungen visualisieren
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    szenarien_namen = df_szenarien['Szenario'].tolist()
    x = np.arange(len(szenarien_namen))
    
    # Fertigungszeit (niedriger = besser)
    ax1 = axes[0]
    colors1 = ['green' if v <= basis_vals['Fertigungszeit'] else 'red' 
              for v in df_szenarien['Fertigungszeit']]
    ax1.bar(x, df_szenarien['Fertigungszeit'], color=colors1, alpha=0.7)
    ax1.axhline(y=basis_vals['Fertigungszeit'], color='black', linestyle='--', label='Basis')
    ax1.set_xticks(x)
    ax1.set_xticklabels(szenarien_namen, rotation=45, ha='right')
    ax1.set_ylabel('Fertigungszeit (min)')
    ax1.set_title('Fertigungszeit')
    ax1.legend()
    
    # Qualität (höher = besser)
    ax2 = axes[1]
    colors2 = ['green' if v >= basis_vals['Qualität'] else 'red' 
              for v in df_szenarien['Qualität']]
    ax2.bar(x, df_szenarien['Qualität'], color=colors2, alpha=0.7)
    ax2.axhline(y=basis_vals['Qualität'], color='black', linestyle='--', label='Basis')
    ax2.set_xticks(x)
    ax2.set_xticklabels(szenarien_namen, rotation=45, ha='right')
    ax2.set_ylabel('Qualitätsscore')
    ax2.set_title('Qualität')
    ax2.legend()
    
    # Energie (niedriger = besser)
    ax3 = axes[2]
    colors3 = ['green' if v <= basis_vals['Energie'] else 'red' 
              for v in df_szenarien['Energie']]
    ax3.bar(x, df_szenarien['Energie'], color=colors3, alpha=0.7)
    ax3.axhline(y=basis_vals['Energie'], color='black', linestyle='--', label='Basis')
    ax3.set_xticks(x)
    ax3.set_xticklabels(szenarien_namen, rotation=45, ha='right')
    ax3.set_ylabel('Energieverbrauch (kWh)')
    ax3.set_title('Energie')
    ax3.legend()
    
    plt.suptitle('Szenario-Vergleich', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Verbesserungspotenzial
    optimiert = df_szenarien[df_szenarien['Szenario'] == 'Voll optimiert'].iloc[0]
    
    print("\n💡 OPTIMIERUNGSPOTENZIAL (Basis → Voll optimiert):")
    print("-" * 50)
    
    zeit_verbesserung = (basis_vals['Fertigungszeit'] - optimiert['Fertigungszeit']) / basis_vals['Fertigungszeit'] * 100
    qual_verbesserung = (optimiert['Qualität'] - basis_vals['Qualität']) / basis_vals['Qualität'] * 100
    energie_verbesserung = (basis_vals['Energie'] - optimiert['Energie']) / basis_vals['Energie'] * 100
    
    print(f"   Fertigungszeit: {zeit_verbesserung:+.1f}% ({basis_vals['Fertigungszeit']:.1f} → {optimiert['Fertigungszeit']:.1f} min)")
    print(f"   Qualität:       {qual_verbesserung:+.1f}% ({basis_vals['Qualität']:.3f} → {optimiert['Qualität']:.3f})")
    print(f"   Energie:        {energie_verbesserung:+.1f}% ({basis_vals['Energie']:.2f} → {optimiert['Energie']:.2f} kWh)")
    
    return df_szenarien

df_szenarien = szenario_analyse(ml_pipeline)

---

## 6. Industrie 4.0 und Zukunftsperspektiven

### Die Vision der Smart Factory

```
┌─────────────────────────────────────────────────────────────────────────┐
│                         SMART FACTORY                                    │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ┌─────────┐    ┌─────────┐    ┌─────────┐    ┌─────────┐             │
│   │ Sensoren│───▶│  Edge   │───▶│  Cloud  │───▶│   KI    │             │
│   │  IoT    │    │Computing│    │Platform │    │ Modelle │             │
│   └─────────┘    └─────────┘    └─────────┘    └────┬────┘             │
│                                                      │                   │
│                                                      ▼                   │
│   ┌─────────┐    ┌─────────┐    ┌─────────┐    ┌─────────┐             │
│   │Maschinen│◀───│Steuerung│◀───│Optimier.│◀───│Vorhersage│            │
│   │ Aktoren │    │ SPS/PLC │    │         │    │Dashboard │            │
│   └─────────┘    └─────────┘    └─────────┘    └─────────┘             │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# ===================================================================
# Industrie 4.0 Trends und Zukunftsperspektiven
# ===================================================================

def zeige_industrie40_trends():
    """
    Zeigt Industrie 4.0 Trends und KI-Anwendungen.
    """
    
    print("\n🏭 INDUSTRIE 4.0 TRENDS UND KI-ANWENDUNGEN")
    print("=" * 70)
    
    trends = {
        'Digital Twin': {
            'icon': '🔄',
            'beschreibung': 'Digitale Zwillinge von Produktionsanlagen',
            'ki_methoden': 'Simulation, Reinforcement Learning, Zeitreihenanalyse',
            'nutzen': 'Virtuelle Optimierung ohne Produktionsrisiko',
            'reifegrad': 3  # 1-5 Skala
        },
        'Predictive Maintenance': {
            'icon': '🔧',
            'beschreibung': 'Vorausschauende Wartung basierend auf Sensordaten',
            'ki_methoden': 'Anomalieerkennung, Survival Analysis, LSTM',
            'nutzen': 'Reduktion ungeplanter Stillstände um 30-50%',
            'reifegrad': 4
        },
        'Qualität 4.0': {
            'icon': '✅',
            'beschreibung': 'KI-basierte Qualitätskontrolle',
            'ki_methoden': 'Computer Vision, CNN, Defect Detection',
            'nutzen': '100% Prüfung, Echtzeit-Feedback',
            'reifegrad': 4
        },
        'Adaptive Fertigung': {
            'icon': '⚙️',
            'beschreibung': 'Selbstoptimierende Produktionsprozesse',
            'ki_methoden': 'Reinforcement Learning, Bayesian Optimization',
            'nutzen': 'Automatische Prozessanpassung',
            'reifegrad': 2
        },
        'Intelligente Planung': {
            'icon': '📊',
            'beschreibung': 'KI-gestützte Produktions- und Kapazitätsplanung',
            'ki_methoden': 'Demand Forecasting, Constraint Optimization',
            'nutzen': 'Bessere Ressourcenauslastung, weniger Bestände',
            'reifegrad': 3
        },
        'Autonome Logistik': {
            'icon': '🤖',
            'beschreibung': 'Selbstfahrende Transportsysteme (AGVs)',
            'ki_methoden': 'Path Planning, SLAM, Multi-Agent RL',
            'nutzen': 'Flexible, skalierbare Intralogistik',
            'reifegrad': 3
        }
    }
    
    for name, details in trends.items():
        print(f"\n{details['icon']} {name}")
        print(f"   📝 {details['beschreibung']}")
        print(f"   🤖 KI-Methoden: {details['ki_methoden']}")
        print(f"   💡 Nutzen: {details['nutzen']}")
        print(f"   📈 Reifegrad: {'●' * details['reifegrad']}{'○' * (5-details['reifegrad'])}")
    
    # Visualisierung: Technologie-Roadmap
    fig, ax = plt.subplots(figsize=(14, 6))
    
    trend_namen = list(trends.keys())
    reifegrade = [trends[t]['reifegrad'] for t in trend_namen]
    
    colors = plt.cm.RdYlGn([r/5 for r in reifegrade])
    
    y_pos = np.arange(len(trend_namen))
    bars = ax.barh(y_pos, reifegrade, color=colors, edgecolor='black')
    
    ax.set_yticks(y_pos)
    ax.set_yticklabels([f"{trends[t]['icon']} {t}" for t in trend_namen])
    ax.set_xlabel('Reifegrad (1=Forschung, 5=Produktiv)')
    ax.set_title('Industrie 4.0 Technologie-Reifegrad', fontsize=14, fontweight='bold')
    ax.set_xlim(0, 5.5)
    
    # Reifegrad-Labels
    for i, (bar, rg) in enumerate(zip(bars, reifegrade)):
        ax.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height()/2,
               f'{rg}/5', va='center', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    return trends

trends = zeige_industrie40_trends()

In [ ]:
# ===================================================================
# Implementierungsstrategie
# ===================================================================

def zeige_implementierungsstrategie():
    """
    Zeigt eine praktische Implementierungsstrategie für KI in der Produktion.
    """
    
    print("\n🎯 IMPLEMENTIERUNGSSTRATEGIE: KI IN DER PRODUKTION")
    print("=" * 70)
    
    phasen = {
        'Phase 1: Foundation (0-6 Monate)': {
            'ziele': 'Grundlagen schaffen',
            'aktivitaeten': [
                'Dateninfrastruktur aufbauen (Data Lake)',
                'IoT-Sensoren an kritischen Anlagen installieren',
                'Datenqualität sicherstellen (Governance)',
                'Team aufbauen / Kompetenzen entwickeln',
                'Use Cases priorisieren (Quick Wins)'
            ],
            'investition': '€€',
            'risiko': 'Niedrig'
        },
        'Phase 2: Pilot (6-12 Monate)': {
            'ziele': 'Proof of Concept',
            'aktivitaeten': [
                'Erste ML-Modelle für ausgewählte Use Cases',
                'Dashboards für Datenvisualisierung',
                'Pilotprojekt: Predictive Quality',
                'ROI messen und dokumentieren',
                'Lessons Learned sammeln'
            ],
            'investition': '€€€',
            'risiko': 'Mittel'
        },
        'Phase 3: Scale (12-24 Monate)': {
            'ziele': 'Skalierung erfolgreicher Piloten',
            'aktivitaeten': [
                'MLOps-Plattform einführen',
                'Modelle in Produktion ausrollen',
                'Integration in bestehende Systeme (ERP, MES)',
                'Mitarbeiter schulen',
                'Weitere Use Cases erschließen'
            ],
            'investition': '€€€€',
            'risiko': 'Mittel-Hoch'
        },
        'Phase 4: Transform (24+ Monate)': {
            'ziele': 'Digitale Transformation',
            'aktivitaeten': [
                'KI als Kernkompetenz etablieren',
                'Autonome Optimierungssysteme',
                'Digital Twins für alle Anlagen',
                'Neue Geschäftsmodelle (as-a-Service)',
                'Kontinuierliche Innovation'
            ],
            'investition': '€€€€€',
            'risiko': 'Hoch (aber kontrolliert)'
        }
    }
    
    for phase, details in phasen.items():
        print(f"\n📍 {phase}")
        print(f"   🎯 Ziel: {details['ziele']}")
        print(f"   💰 Investition: {details['investition']}")
        print(f"   ⚠️ Risiko: {details['risiko']}")
        print(f"   📋 Aktivitäten:")
        for akt in details['aktivitaeten']:
            print(f"      • {akt}")
    
    # Erfolgsfaktoren
    print("\n" + "="*70)
    print("💡 KRITISCHE ERFOLGSFAKTOREN")
    print("=" * 70)
    
    erfolgsfaktoren = [
        ('Management Commitment', 'Top-Down Unterstützung und Vision'),
        ('Datenqualität', 'Garbage In = Garbage Out'),
        ('Change Management', 'Mitarbeiter mitnehmen, nicht ersetzen'),
        ('Schrittweise Vorgehen', 'Kleine Erfolge statt Big Bang'),
        ('Cross-funktionale Teams', 'IT + OT + Fachbereich'),
        ('Realistische Erwartungen', 'KI ist kein Allheilmittel'),
        ('Kontinuierliches Lernen', 'Modelle müssen gepflegt werden')
    ]
    
    for i, (faktor, erklaerung) in enumerate(erfolgsfaktoren, 1):
        print(f"   {i}. {faktor}")
        print(f"      → {erklaerung}")

zeige_implementierungsstrategie()

---

## 7. Aufgaben für Studierende

Bearbeiten Sie die folgenden Aufgaben, um Ihr Verständnis der KI-Integration in der Produktion zu vertiefen.

### ✏️ Aufgabe 1: Modellverbesserung

Das aktuelle Modell zur Qualitätsprognose hat einen R²-Score von ca. 0.85. Verbessern Sie das Modell:

1. **Feature Engineering**: Erstellen Sie neue Features (z.B. Interaktionen, Polynome)
2. **Hyperparameter-Tuning**: Optimieren Sie die Modellparameter
3. Vergleichen Sie das verbesserte Modell mit dem Basis-Modell

In [ ]:
# Ihre Lösung für Aufgabe 1:
# ===========================

# TODO: Feature Engineering implementieren
# TODO: Hyperparameter-Tuning durchführen
# TODO: Modelle vergleichen



In [ ]:
# ===================================================================
# LÖSUNG Aufgabe 1 (zur Selbstkontrolle)
# ===================================================================

print("📝 LÖSUNG AUFGABE 1: Modellverbesserung")
print("=" * 65)

from sklearn.model_selection import GridSearchCV

# 1. Feature Engineering
print("\n1️⃣ FEATURE ENGINEERING")
print("-" * 40)

df_enhanced = df.copy()

# Neue Features erstellen
df_enhanced['Wartung_x_Material'] = df_enhanced['Tage_seit_Wartung'] * df_enhanced['Materialqualitaet']
df_enhanced['Erfahrung_x_Material'] = df_enhanced['Operator_Erfahrung_Jahre'] * df_enhanced['Materialqualitaet']
df_enhanced['Temperatur_Abweichung'] = abs(df_enhanced['Temperatur_C'] - 22)  # Abweichung von Optimal
df_enhanced['Wartung_kritisch'] = (df_enhanced['Tage_seit_Wartung'] > 20).astype(int)

print("Neue Features erstellt:")
print("   • Wartung_x_Material (Interaktion)")
print("   • Erfahrung_x_Material (Interaktion)")
print("   • Temperatur_Abweichung (vom Optimum)")
print("   • Wartung_kritisch (binär: >20 Tage)")

# 2. Daten vorbereiten
feature_cols_enhanced = ['Produkttyp', 'Maschine', 'Schicht',
                        'Temperatur_C', 'Luftfeuchtigkeit_pct',
                        'Materialqualitaet', 'Operator_Erfahrung_Jahre',
                        'Tage_seit_Wartung', 
                        'Wartung_x_Material', 'Erfahrung_x_Material',
                        'Temperatur_Abweichung', 'Wartung_kritisch']

X_enh = df_enhanced[feature_cols_enhanced].copy()

# Encoding
for col in ['Produkttyp', 'Maschine', 'Schicht']:
    le = LabelEncoder()
    X_enh[col] = le.fit_transform(X_enh[col])

y_qual = df_enhanced['Qualitaetsscore']

X_train, X_test, y_train, y_test = train_test_split(X_enh, y_qual, test_size=0.2, random_state=42)

# 3. Hyperparameter-Tuning
print("\n2️⃣ HYPERPARAMETER-TUNING")
print("-" * 40)

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 15, 20],
    'min_samples_split': [2, 5]
}

rf = RandomForestRegressor(random_state=42, n_jobs=-1)

grid_search = GridSearchCV(rf, param_grid, cv=3, scoring='r2', n_jobs=-1)
grid_search.fit(X_train, y_train)

print(f"Beste Parameter: {grid_search.best_params_}")
print(f"Bester CV R²: {grid_search.best_score_:.4f}")

# 4. Vergleich
print("\n3️⃣ MODELLVERGLEICH")
print("-" * 40)

# Basis-Modell (ohne Feature Engineering)
basis_r2 = results_qual[best_qual]['r2']
basis_mae = results_qual[best_qual]['mae']

# Verbessertes Modell
best_model = grid_search.best_estimator_
y_pred_enhanced = best_model.predict(X_test)
enhanced_r2 = r2_score(y_test, y_pred_enhanced)
enhanced_mae = mean_absolute_error(y_test, y_pred_enhanced)

print(f"{'Metrik':<15} {'Basis-Modell':>15} {'Verbessertes Modell':>20}")
print("-" * 55)
print(f"{'R²':<15} {basis_r2:>15.4f} {enhanced_r2:>20.4f}")
print(f"{'MAE':<15} {basis_mae:>15.4f} {enhanced_mae:>20.4f}")

verbesserung_r2 = (enhanced_r2 - basis_r2) / basis_r2 * 100
verbesserung_mae = (basis_mae - enhanced_mae) / basis_mae * 100

print("\n📈 VERBESSERUNG:")
print(f"   R²: {verbesserung_r2:+.1f}%")
print(f"   MAE: {verbesserung_mae:+.1f}%")

### ✏️ Aufgabe 2: ROI-Berechnung

Berechnen Sie den Return on Investment (ROI) für die KI-Implementierung:

**Annahmen:**
- Jährliche Produktion: 100.000 Einheiten
- Kosten pro defekte Einheit: 50 €
- Aktuelle Ausschussrate: 4%
- Mit KI-Optimierung: 2.5% (Reduktion um 37.5%)
- KI-Implementierungskosten: 150.000 € (einmalig)
- Jährliche Wartungskosten: 30.000 €

**Berechnen Sie:**
1. Jährliche Einsparung durch Ausschussreduktion
2. ROI im ersten Jahr
3. Amortisationszeit (Payback Period)

In [ ]:
# Ihre Lösung für Aufgabe 2:
# ===========================

# TODO: Einsparung berechnen
# TODO: ROI berechnen
# TODO: Amortisationszeit berechnen



In [ ]:
# ===================================================================
# LÖSUNG Aufgabe 2 (zur Selbstkontrolle)
# ===================================================================

print("📝 LÖSUNG AUFGABE 2: ROI-Berechnung")
print("=" * 65)

# Parameter
produktion_jahr = 100000
kosten_defekt = 50
ausschuss_vorher = 0.04
ausschuss_nachher = 0.025
ki_investition = 150000
ki_wartung_jahr = 30000

print("\n📋 PARAMETER:")
print("-" * 40)
print(f"   Produktion/Jahr:      {produktion_jahr:>12,} Einheiten")
print(f"   Kosten/Defekt:        {kosten_defekt:>12} €")
print(f"   Ausschuss vorher:     {ausschuss_vorher*100:>12.1f}%")
print(f"   Ausschuss nachher:    {ausschuss_nachher*100:>12.1f}%")
print(f"   KI-Investition:       {ki_investition:>12,} €")
print(f"   KI-Wartung/Jahr:      {ki_wartung_jahr:>12,} €")

# Berechnungen
print("\n📊 BERECHNUNGEN:")
print("-" * 40)

# 1. Ausschusskosten vorher und nachher
defekte_vorher = produktion_jahr * ausschuss_vorher
defekte_nachher = produktion_jahr * ausschuss_nachher
kosten_vorher = defekte_vorher * kosten_defekt
kosten_nachher = defekte_nachher * kosten_defekt

print(f"   Defekte vorher:       {defekte_vorher:>12,.0f} Stück")
print(f"   Defekte nachher:      {defekte_nachher:>12,.0f} Stück")
print(f"   Einsparung Defekte:   {defekte_vorher - defekte_nachher:>12,.0f} Stück")

# 2. Jährliche Einsparung
brutto_einsparung = kosten_vorher - kosten_nachher
netto_einsparung = brutto_einsparung - ki_wartung_jahr

print(f"\n   Ausschusskosten vorher: {kosten_vorher:>10,.0f} €/Jahr")
print(f"   Ausschusskosten nachher: {kosten_nachher:>9,.0f} €/Jahr")
print(f"   Brutto-Einsparung:      {brutto_einsparung:>10,.0f} €/Jahr")
print(f"   - KI-Wartungskosten:    {ki_wartung_jahr:>10,.0f} €/Jahr")
print(f"   ─────────────────────────────────────")
print(f"   Netto-Einsparung:       {netto_einsparung:>10,.0f} €/Jahr")

# 3. ROI berechnen
print("\n📈 ROI-ANALYSE:")
print("-" * 40)

# ROI Jahr 1 (inkl. Investition)
gewinn_jahr1 = netto_einsparung - ki_investition
roi_jahr1 = gewinn_jahr1 / ki_investition * 100

print(f"   Jahr 1:")
print(f"      Netto-Einsparung:    {netto_einsparung:>10,.0f} €")
print(f"      - Investition:       {ki_investition:>10,.0f} €")
print(f"      = Gewinn/Verlust:    {gewinn_jahr1:>+10,.0f} €")
print(f"      ROI Jahr 1:          {roi_jahr1:>+10.1f}%")

# Amortisationszeit
payback_monate = ki_investition / (netto_einsparung / 12)
payback_jahre = payback_monate / 12

print(f"\n   Amortisation:")
print(f"      Payback Period:      {payback_monate:>10.1f} Monate")
print(f"                           {payback_jahre:>10.1f} Jahre")

# 5-Jahres-Projektion
print("\n📊 5-JAHRES-PROJEKTION:")
print("-" * 50)
print(f"{'Jahr':<6} {'Cashflow':>12} {'Kumuliert':>12} {'ROI kum.':>12}")
print("-" * 50)

kumuliert = 0
for jahr in range(1, 6):
    if jahr == 1:
        cashflow = netto_einsparung - ki_investition
    else:
        cashflow = netto_einsparung
    kumuliert += cashflow
    roi_kum = kumuliert / ki_investition * 100
    print(f"{jahr:<6} {cashflow:>+12,.0f} € {kumuliert:>+12,.0f} € {roi_kum:>+11.0f}%")

print("\n💡 FAZIT:")
print(f"   Die KI-Investition amortisiert sich nach {payback_monate:.0f} Monaten.")
print(f"   Nach 5 Jahren beträgt der kumulierte ROI {roi_kum:.0f}%.")

### ✏️ Aufgabe 3: Use Case Entwicklung

Entwickeln Sie einen eigenen KI-Use-Case für die Produktion:

1. **Problemdefinition**: Welches Problem soll gelöst werden?
2. **Datenquellen**: Welche Daten werden benötigt?
3. **KI-Methode**: Welche Algorithmen sind geeignet?
4. **Nutzen**: Welcher Mehrwert wird geschaffen?
5. **Implementierung**: Wie könnte die Umsetzung aussehen?

In [ ]:
# Ihre Lösung für Aufgabe 3:
# ===========================

# TODO: Use Case dokumentieren
# Optional: Prototyp implementieren



In [ ]:
# ===================================================================
# LÖSUNG Aufgabe 3: Beispiel Use Case (zur Selbstkontrolle)
# ===================================================================

print("📝 LÖSUNG AUFGABE 3: Use Case - Predictive Maintenance")
print("=" * 70)

use_case = {
    'titel': 'Predictive Maintenance für CNC-Maschinen',
    'problem': '''
    Ungeplante Maschinenausfälle führen zu:
    • Produktionsstillständen (Ø 4h pro Ausfall)
    • Eilreparaturkosten (+50% Aufschlag)
    • Qualitätsproblemen vor Ausfall
    • Lieferverzögerungen und Konventionalstrafen
    
    Ziel: Ausfälle 24-72h im Voraus vorhersagen
    ''',
    'datenquellen': [
        'Vibrationssensoren (Beschleunigung, Frequenzspektrum)',
        'Temperatursensoren (Spindel, Kühlmittel)',
        'Stromaufnahme (Antriebe)',
        'Akustische Sensoren (Geräuschpegel)',
        'Wartungshistorie (SAP PM)',
        'Betriebsstunden und Lastzyklen'
    ],
    'ki_methoden': [
        'Anomalieerkennung: Isolation Forest, Autoencoder',
        'Zeitreihenanalyse: LSTM, Transformer',
        'Survival Analysis: Cox Regression',
        'Klassifikation: Random Forest, XGBoost'
    ],
    'nutzen': {
        'Reduktion ungeplanter Stillstände': '60-80%',
        'Reduktion Wartungskosten': '20-30%',
        'Erhöhung Anlagenverfügbarkeit': '5-10%',
        'Verbesserung Produktqualität': '10-15%'
    },
    'implementierung': [
        'Monat 1-2: Sensorik installieren, Datenerfassung starten',
        'Monat 3-4: Historische Daten sammeln, EDA durchführen',
        'Monat 5-6: Erste ML-Modelle trainieren und evaluieren',
        'Monat 7-8: Dashboard entwickeln, Pilotbetrieb',
        'Monat 9-12: Rollout, Integration in Wartungsplanung'
    ]
}

print(f"\n🎯 USE CASE: {use_case['titel']}")
print("\n📋 PROBLEMSTELLUNG:")
print(use_case['problem'])

print("\n📊 DATENQUELLEN:")
for source in use_case['datenquellen']:
    print(f"   • {source}")

print("\n🤖 KI-METHODEN:")
for method in use_case['ki_methoden']:
    print(f"   • {method}")

print("\n💰 ERWARTETER NUTZEN:")
for nutzen, wert in use_case['nutzen'].items():
    print(f"   • {nutzen}: {wert}")

print("\n📅 IMPLEMENTIERUNGSPLAN:")
for schritt in use_case['implementierung']:
    print(f"   • {schritt}")

---

## 8. Zusammenfassung und Ausblick

### Was haben wir gelernt?

| Thema | Kernaussage |
|-------|-------------|
| **ML-Pipeline** | Strukturierter Workflow von Daten bis Deployment |
| **Feature Engineering** | Domänenwissen in Features übersetzen |
| **Modellvergleich** | Verschiedene Algorithmen evaluieren, besten auswählen |
| **Feature Importance** | Verstehen, was das Modell gelernt hat |
| **Szenario-Analyse** | Modelle für What-If-Analysen nutzen |
| **Industrie 4.0** | KI als Enabler für Smart Manufacturing |
| **Implementierung** | Schrittweise Vorgehensweise für Erfolg |

### Die Zukunft der intelligenten Produktion

```
       HEUTE                    MORGEN                   ÜBERMORGEN
    ┌──────────┐             ┌──────────┐             ┌──────────┐
    │Deskriptiv│   ────▶     │Prädiktiv │   ────▶     │Präskriptiv│
    │"Was ist  │             │"Was wird │             │"Was soll │
    │passiert?"│             │passieren?│             │ich tun?" │
    └──────────┘             └──────────┘             └──────────┘
    
    Dashboards              ML-Modelle              Autonome
    Reports                 Forecasting             Optimierung
    BI-Tools                Alerting                Selbststeuerung
```

In [ ]:
# ===================================================================
# Abschluss und Checkliste
# ===================================================================

print("\n" + "="*70)
print("📋 CHECKLISTE - KI-Integration in der Produktion")
print("="*70)

checkliste = [
    ("Produktionsdaten verstehen", 
     "EDA durchführen, Zusammenhänge erkennen, Datenqualität prüfen"),
    ("ML-Pipeline aufbauen", 
     "Preprocessing, Train/Test Split, Cross-Validation"),
    ("Modelle trainieren und vergleichen", 
     "Random Forest, Gradient Boosting, Metriken: MAE, R²"),
    ("Feature Importance interpretieren", 
     "Wichtige Einflussfaktoren identifizieren"),
    ("Szenario-Analysen durchführen", 
     "What-If-Analysen für Optimierungsempfehlungen"),
    ("Industrie 4.0 Trends kennen", 
     "Digital Twin, Predictive Maintenance, Quality 4.0"),
    ("Implementierungsstrategie entwickeln", 
     "Phasenmodell: Foundation → Pilot → Scale → Transform"),
    ("ROI berechnen können", 
     "Kosten-Nutzen-Analyse, Amortisationszeit")
]

for i, (punkt, beschreibung) in enumerate(checkliste, 1):
    print(f"\n☐ {i}. {punkt}")
    print(f"   → {beschreibung}")

print("\n" + "="*70)
print("🎓 HERZLICHEN GLÜCKWUNSCH!")
print("="*70)
print("\nSie haben alle Notebooks der Vorlesung")
print("'Intelligente Produktionsplanung und KI' erfolgreich abgeschlossen!")
print("\n🚀 Jetzt sind Sie bereit, KI in der Praxis einzusetzen!")
print("\n📚 Weiterführende Ressourcen:")
print("   • Scikit-learn Dokumentation: scikit-learn.org")
print("   • Kaggle Kurse: kaggle.com/learn")
print("   • Google ML Crash Course: developers.google.com/machine-learning")
print("="*70)